In [ ]:
import networkx as nx
import cantera as ct
import numpy as np
from bokeh.models import Circle, MultiLine, HoverTool
from bokeh.plotting import figure, from_networkx, show, output_notebook
from networkx.drawing.nx_agraph import graphviz_layout


In [ ]:

mechanism_file = "/home/moon/autokmc/rmg2kmc/examples/co_oxidation/chem_annotated.cti"

In [ ]:
T = 1000.0


gas = ct.Solution(mechanism_file, "gas")
surf = ct.Interface(mechanism_file, "surface1", [gas])

species_list = surf.species() + gas.species()
reaction_list_total = surf.reactions() + gas.reactions()

reaction_weights = [rxn.rate(T) for rxn in reaction_list_total]

In [ ]:
reaction_list = reaction_list_total
hide_inerts = True
inerts = set(['Ar', 'Ne', 'N2'])
overall_reactants = set(['O2(2)', 'CO(4)'])
overall_products = set(['CO2(3)'])

G = nx.Graph()

for i, species in enumerate(species_list):
    if hide_inerts:
        if species.name in inerts:
            continue
#         G.add_node(species.name, name=species.name)
        G.add_node(species.name, name=species.name, thermo=species.thermo.h(T))

node_color_map = []
node_attr = {}
for node in G:
    if node in overall_reactants:
#         node_color_map.append('red')
        node_color_map.append('#FF0000')
        node_attr[node] = 'red'
    elif node in overall_products:
#         node_color_map.append('green')
        node_color_map.append('#00FF00')
        node_attr[node] = 'green'
    else:
#         node_color_map.append('blue')
        node_color_map.append('#0000FF')
        node_attr[node] = 'grey'

nx.set_node_attributes(G, node_attr, name="node_color")

edge_weights = {}
for reaction in reaction_list:
    for product_name in reaction.products:
        for reactant_name in reaction.reactants:
#             G.add_edge(product_name, reactant_name, color='black')
#             edge_weights.append(np.log(reaction.rate(T)))
            edge_weights[product_name, reactant_name] = np.log(reaction.rate(T))
            G.add_edge(product_name, reactant_name, color='black')
nx.set_edge_attributes(G, edge_weights, name="edge_rate")

In [ ]:
output_notebook()
edge_attrs = {}


# don't forget the seed, or it'll draw it differently every time
plot = figure(width=400, height=400,
              x_axis_location=None, y_axis_location=None, toolbar_location=None,
              title="Graph Interaction Demo", background_fill_color="#efefef",
              tooltips="name: @name, G: @thermo, edge_rate: @edge_rate")
# plot = figure(width=400, height=400, x_range=(-1.2, 1.2), y_range=(-1.2, 1.2),
#               x_axis_location=None, y_axis_location=None, toolbar_location=None,
#               title="Graph Interaction Demo", background_fill_color="#efefef",
#               tooltips="name: @name, G: @thermo")
# # plot.grid.grid_line_color = None

pos1 = nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')

graph_renderer = from_networkx(G, pos1)
# graph_renderer = from_networkx(G, nx.spring_layout, seed=40, scale=1, center=(0, 0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
plot.renderers.append(graph_renderer)

# graph_renderer.edge_renderer.glyph = Circle(size=15, fill_color="node_color")
# graph_renderer.edge_renderer.glyph = Circle(size=15)

# graph_renderer.edge_renderer.glyph.line_width = {'field': 'edge_rate'}
# hover_edges = HoverTool(
#     tooltips=[('edge_rate','@edge_rate')],
#     renderers=[graph_renderer.edge_renderer], line_policy="interp"
# )
# plot.add_tools(hover_edges)

show(plot)

In [ ]:
species_list[4].thermo.h(1200)

In [ ]:
reaction_list[2].reactants

In [ ]:
delta_H = 0
reaction_index = 2
for key in reaction_list[reaction_index].products:
    try:
        print(surf.species(key).thermo.h(T))
        
    except:
        print(gas.species(key).thermo.h(T))


for key in reaction_list[reaction_index].reactants:
    try:
        print(surf.species(key).thermo.h(T))
    except:
        print(gas.species(key).thermo.h(T))


In [ ]:
pos1

In [ ]:
nx.spring_layout(G)

In [ ]:
graph_renderer.edge_renderer

In [ ]:
node_attr

In [ ]:
G['',1]

In [ ]:
reaction_list[0]

In [ ]:
# dir(surf)
print(surf.reactions()[0])
K = surf.equilibrium_constants[0]
print(K)
k_fwd = surf.reactions()[0].rate(294)
k_rev = k_fwd / K
print(f'k_fwd:\t{k_fwd}')
print(f'k_rev:\t{k_rev}')
# len(surf.equilibrium_constants)
len(surf.reactions())
# dir(reaction_list[0])

In [ ]:
# Wintterlin